In [28]:
import sys
sys.path.insert(0, '/home/brendalf/Documents/projects/gan-attack/src')

In [29]:
import os
import torch
import torchvision

import numpy as np
import torch.nn as nn

import torchvision
from torchvision.transforms import Compose, Resize, Normalize, ToTensor, RandomCrop, RandomHorizontalFlip
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

In [30]:
from target.train import train_network
from target.evaluate import evaluate_network

from target.custom import CustomNN

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz + nl, ngf*4, 4, 1, 0),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(),
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(),
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1),
            nn.BatchNorm2d(ngf),
            nn.ReLU(),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, input, label):
        x = torch.cat((input, label), dim=1).unsqueeze(2).unsqueeze(3)
        return self.main(x)

In [33]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
                 
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2)
        )

        self.clf = nn.Sequential(
            nn.Linear(
                in_features=ndf * 4 * 4 * 4,
                out_features=nl,
                bias=True
            ),
            torch.nn.Softmax(dim=1)
        )

        self.police = torch.nn.Sequential(
            torch.nn.Conv2d(ndf * 4, 1, 4, 1, 0),
            torch.nn.Sigmoid()
        )

    def forward(self, input):
        features = self.main(input)
        valid = self.police(features).view(-1, 1)
        clf = self.clf(features.view(features.shape[0], -1))
        return valid, clf

In [34]:
netG = torch.load('../models/adversary/g_cifar10_500img_modeseeking.pth').to(device)
netD = torch.load('../models/adversary/d_cifar10_500img_modeseeking.pth').to(device)

In [35]:
num_classes = 10
training_size = 1000

In [36]:
def encodeOneHot(labels):
    ret = torch.FloatTensor(labels.shape[0], num_classes)
    ret.zero_()
    ret.scatter_(dim=1, index=labels.view(-1, 1), value=1)
    return ret

In [37]:
for n in np.arange(0, num_classes):
    print(f"Generating images from class {n}")
    output_path = os.path.join('../data/copycat_gan_vgg19/', str(n))
    
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    
    for i in np.arange(0, 10):
        X = torch.randn(training_size, 100).to(device)
        y = encodeOneHot(
            torch.ones(training_size, dtype=torch.int64) * n
        ).to(device)
        
        generated = netG(X, y)
        
        for id, image in enumerate(generated):
            new_id = (training_size * i) + id
            torchvision.utils.save_image(
                image,
                fp=os.path.join(output_path, f'{new_id}.png')    
            )

Generating images from class 0
Generating images from class 1
Generating images from class 2
Generating images from class 3
Generating images from class 4
Generating images from class 5
Generating images from class 6
Generating images from class 7
Generating images from class 8
Generating images from class 9


In [38]:
imagefolder = torchvision.datasets.ImageFolder(
    '../data/copycat_gan_vgg19/',
    transform=Compose([
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
)

In [39]:
dataloader = torch.utils.data.DataLoader(
    imagefolder,
    batch_size=16,
    num_workers=2
)

In [40]:
import torch.backends.cudnn as cudnn

cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [41]:
net = VGG('VGG19')

In [42]:
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

net = net.to(device)

In [43]:
epochs = 50

In [44]:
output_path = "../models/adversary/copycat.vgg19.pth"

In [45]:
transform_test = Compose([
    ToTensor(),
    Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

Files already downloaded and verified


In [46]:
model = net

In [47]:
import tqdm
import torch
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#best device available
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in dataloader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    else:
        correct = 0
        total = 0
        val_loss = 0.0

        with torch.no_grad(): # turn off grad
            model.eval() # network in evaluation mode

            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    acc = (100 * correct / total)
    print('E: {}/{} [train_loss: {:.3f}, val_loss: {:.3f}, val_acc: {:.3f}]'.format(epoch+1, epochs, running_loss, val_loss, acc))
    #print('E: {}/{} [train_loss: {:.3f}]'.format(epoch+1, epochs, running_loss))

E: 1/50 [train_loss: 1995.153, val_loss: 52412.984, val_acc: 10.000]
E: 2/50 [train_loss: 1545.293, val_loss: 91899.405, val_acc: 10.000]
E: 3/50 [train_loss: 1445.294, val_loss: 73201.231, val_acc: 10.000]
E: 4/50 [train_loss: 1449.042, val_loss: 46025.673, val_acc: 10.000]
E: 5/50 [train_loss: 1390.694, val_loss: 5162.335, val_acc: 10.000]
E: 6/50 [train_loss: 1320.643, val_loss: 6369.206, val_acc: 10.000]
E: 7/50 [train_loss: 1765.901, val_loss: 8417.816, val_acc: 10.000]
E: 8/50 [train_loss: 1499.738, val_loss: 3062.364, val_acc: 10.000]
E: 9/50 [train_loss: 1290.079, val_loss: 3652.005, val_acc: 10.000]
E: 10/50 [train_loss: 1141.796, val_loss: 4182.205, val_acc: 10.000]
E: 11/50 [train_loss: 1208.060, val_loss: 4060.819, val_acc: 10.000]
E: 12/50 [train_loss: 1039.326, val_loss: 8480.868, val_acc: 10.000]
E: 13/50 [train_loss: 1067.935, val_loss: 4172.867, val_acc: 10.000]
E: 14/50 [train_loss: 977.466, val_loss: 2522.373, val_acc: 10.000]
E: 15/50 [train_loss: 1189.932, val_loss

In [48]:
evaluate_network(model, testloader)

  0%|          | 0/313 [00:00<?, ?it/s]

Evaluating model...


100%|██████████| 313/313 [00:03<00:00, 96.13it/s] 

Accuracy: 12 %


12.05